1. setup dataset with train (seen 80%) and val (seen10%+unseen10%).
2. train arcface with trainset.
3. monitor roc with val.

In [1]:
batch_size = 256
TRAIN = False
load_epoch = 20


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import albumentations as alb
import os
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,models
from tqdm import tqdm_notebook as tqdm
from torch.optim import lr_scheduler

import sklearn.metrics

In [3]:
df_train = pd.read_csv('./train_v2.csv')
n_fold = 5
for fold in range(n_fold):
    train_idx = np.where((df_train['fold'] != fold) & (df_train['unseen'] == 0))[0]
    valid_idx = np.where((df_train['fold'] == fold) | (df_train['unseen'] != 0))[0]
    val_seen = np.where((df_train['fold'] == fold))[0]
    val_unseen = np.where((df_train['unseen'] != 0))[0]

df_train2 = df_train.loc[train_idx].reset_index(drop=True)
df_dev = df_train.loc[val_seen].reset_index(drop=True)
df_unseen = df_train.loc[val_unseen].reset_index(drop=True)

In [4]:
from autoaugument import ImageNetPolicy
from PIL import Image

RATIO = 1
SIZE = 236
SIZE2 = 128

class GraphemeDataset(Dataset):
    def __init__(self, label, data_aug=False, _type='train'):
        self.label = label
        self.data_aug_func = train_aug_trasforms = alb.Compose([
                alb.RandomBrightnessContrast(0.05, 0.05, p=0.3),
                alb.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
                alb.RandomSizedCrop(min_max_height=(int(70*RATIO), int(127*RATIO)), height=128, width=128, 
                                    w2h_ratio=1, interpolation=1, p=0.2),
                alb.Rotate(limit=20, p=0.4),
                alb.Cutout(num_holes=8, max_h_size=int(18*RATIO), max_w_size=int(18*RATIO), p=0.4),
                
                #alb.RandomResizedCrop(p=0.3),
            ], p=1)
        self.data_aug = data_aug
        
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self,idx):
        label1 = self.label.vowel_diacritic.values[idx]
        label2 = self.label.grapheme_root.values[idx]
        label3 = self.label.consonant_diacritic.values[idx]
        image = cv2.imread("train{}/".format(SIZE)+self.label.image_id.values[idx]+".png")
        image = cv2.resize(image, (SIZE2,SIZE2))

        imagetrans = self.data_aug_func(image=image)["image"]
        
        # to torch formats
        image = np.array(image)/255
        image = np.transpose(image, (2,0,1))
        imagetrans = np.array(imagetrans)/255
        imagetrans = np.transpose(imagetrans, (2,0,1))
        return image, imagetrans, label1,label2,label3

In [5]:
train_image = GraphemeDataset(df_train2, data_aug=True)
train_loader = torch.utils.data.DataLoader(train_image,batch_size=batch_size,shuffle=True, num_workers=8)
dev_image = GraphemeDataset(df_dev)
dev_loader = torch.utils.data.DataLoader(dev_image,batch_size=batch_size,shuffle=True, num_workers=8)
unseen_image = GraphemeDataset(df_unseen)
unseen_loader = torch.utils.data.DataLoader(unseen_image, batch_size=batch_size,shuffle=True, num_workers=8)

## models

In [6]:
import pretrainedmodels
model_name = "resnet34"
basemodel = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
basemodel = nn.Sequential(*list(basemodel.children())[:-2])

In [7]:
from metrics import ArcMarginProduct
class mymodel(nn.Module):
    def __init__(self):
        super(mymodel, self).__init__()
        self.features = basemodel
        #self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1, bias=False)
        if model_name == "resnet34" or model_name == "resnet18":
            num_ch = 512
        else:
            num_ch = 2048
        # vowel_diacritic       
        self.fc1 = nn.Conv2d(num_ch, 1, 1)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        
    def forward(self, x):
        # extract features
        #x = self.conv1(x)
        x = self.features(x)
        x = self.avgpool(x).squeeze(2).squeeze(2)

        #x1 = self.arcface(x).squeeze(2).squeeze(2)
        
        return x

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = mymodel()
model = model.to(device)

In [9]:
metric_fc = ArcMarginProduct(512, 2, s=100, m=0.5, easy_margin=False).to(device)
optimizer = torch.optim.Adam([{'params': model.parameters()}, {'params': metric_fc.parameters()}],
                                     lr=1e-3, weight_decay=1e-4)
criterion = torch.nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True, min_lr=1e-3*1e-3, factor=0.1)

## train

In [10]:
def train(epoch):
    model.train()
    print('epochs {}/{} '.format(epoch+1,epochs))
    running_loss = 0.0
    running_acc = 0.0
    acc1 = 0.0
    acc2 = 0.0
    acc3 = 0.0
    t = tqdm(train_loader)
    
    for idx, (inputs,inputstrans,_,_,_) in enumerate(t):       
        # send to gpu
        inputs = inputs.to(device)
        labels1 = torch.zeros(inputs.size()[0]).to(device).long() #.unsqueeze(1)
        
        # set opt
        optimizer.zero_grad()
        
        # run model
        feat = model(inputs.float())
        outputs = metric_fc(feat, labels1)
        #print(feat.size())
        #print(labels1.size())
        #print(outputs.size())
        loss = criterion(outputs,labels1)
        running_loss += loss
        
        loss.backward()
        optimizer.step()
        
        t.set_description(f't (l={running_loss/(idx+1):.3f})')
    # save logs
    log_epoch = {'epoch': epoch+1, 'lr': optimizer.state_dict()['param_groups'][0]['lr'],
                     'loss': running_loss/len(train_loader)}
    logs_train.append(log_epoch)
    df = pd.DataFrame(logs_train)
    df.to_csv("log/log_output_train_arc.csv")

In [11]:
def eval(epoch):
    model.eval()
    print('epochs {}/{} '.format(epoch+1,epochs))
    running_loss = 0.0
    running_acc = 0.0
    acc1 = 0.0
    acc2 = 0.0
    acc3 = 0.0
    t = tqdm(dev_loader)
    
    for idx, (inputs,inputstrans,_,_,_) in enumerate(t):       
        # send to gpu
        inputs = inputs.to(device)
        labels1 = torch.zeros(inputs.size()[0]).to(device).long() #.unsqueeze(1)
        
        # set opt
        optimizer.zero_grad()
        
        # run model
        with torch.no_grad():
            feat = model(inputs.float())
            outputs = metric_fc(feat, labels1)
        #print(outputs)
        #print(feat.size())
        #print(labels1.size())
        #print(outputs.size())
        loss = criterion(outputs,labels1)
        running_loss += loss
        
        #loss.backward()
        #optimizer.step()
        
        t.set_description(f't (l={running_loss/(idx+1):.3f})')
    # save logs
    log_epoch = {'epoch': epoch+1, 'lr': optimizer.state_dict()['param_groups'][0]['lr'],
                     'loss': running_loss/len(dev_loader)}
    logs_eval.append(log_epoch)
    df = pd.DataFrame(logs_eval)
    df.to_csv("log/log_output_eval_arc.csv")

In [12]:
epochs = 50
import gc
logs_eval = [];logs_train = [];
model.load_state_dict(torch.load('./models/arcface_saved_weights.pth'))
if TRAIN:
    for epoch in range(epochs):
        # GC
        torch.cuda.empty_cache()
        gc.collect()
        #train(epoch)
        eval(epoch)
        train(epoch)
        torch.save(model.state_dict(), './models/{}arcface_saved_weights.pth'.format(epoch))
        
else:
    model.load_state_dict(torch.load('./models/{}arcface_saved_weights.pth'.format(load_epoch)))

## test model

In [13]:
def inference(dataloader):
    t = tqdm(dataloader)  
    for idx, (inputs,_,_,_,_) in enumerate(t):       
        # send to gpu
        inputs = inputs.to(device)
        # run model
        with torch.no_grad():
            out = model(inputs.float())
        if idx  == 0:
            outs = out
        else:
            outs = torch.cat((outs, out))

    return outs.cpu().numpy()

In [14]:
# get feature matrix
train_results = inference(train_loader)
test_seen = inference(dev_loader)
test_unseen = inference(unseen_loader)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [15]:
print(train_results.shape)
print(test_seen.shape)
print(test_unseen.shape)

(154610, 512)
(38652, 512)
(7578, 512)


In [ ]:
# get distance.
def cosin_metric(x1, x2):
    return np.dot(x1, x2)# / (np.linalg.norm(x1) * np.linalg.norm(x2))

nmax = 100

test_seen = np.clip(np.mean(cosin_metric(train_results, test_seen.T), axis=0), 0, nmax)
test_unseen = np.clip(np.mean(cosin_metric(train_results, test_unseen.T), axis=0), 0, nmax)

In [ ]:
# print scatter plot of same 1s
import matplotlib.pyplot as plt
plt.plot(test_seen)
plt.show()
plt.plot(test_unseen)
plt.show()

## plot roc curve

In [ ]:
predict_y = np.concatenate([test_seen, test_unseen])
samelabel = np.zeros_like(test_seen)
diflabel = np.ones_like(test_unseen)
test_y = np.concatenate([samelabel, diflabel])

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np

# FPR, TPR(, しきい値) を算出
fpr, tpr, thresholds = metrics.roc_curve(test_y, predict_y)
# ついでにAUCも
auc = metrics.auc(fpr, tpr)

# ROC曲線をプロット
plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
plt.legend()
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)